In [177]:
import pandas as pd
import requests
import csv
import time
import datetime
import config as config

In [178]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [179]:
url = "https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=iu:1&count=5"
payload= {}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)

print(response.json()["time"])
print(response.json()["departures"][0]["stop_id"])
print(response.json()["departures"][0]["headsign"])
print(response.json()["departures"][1]["trip"]["direction"])
print(response.json()["departures"][0]["location"])
print("Sceduled: " + str(response.json()["departures"][1]["scheduled"]))
print("Expected: " + str(response.json()["departures"][1]["expected"]))
print(str(response.json()["departures"][1]["expected_mins"]) + " mins")

2021-10-23T18:57:15-05:00
IU:1
130N Silver
North
{'lat': 40.105893, 'lon': -88.22875}
Sceduled: 2021-10-23T18:56:00-05:00
Expected: 2021-10-23T18:58:57-05:00
2 mins


In [327]:
# the time it says the bus will arrive vs. when it actually arrived

# check scheduled vs. expected time -> initial expected time difference
# then, send requests for current time as expected time gets closer to 0 --> -2 < 0 < 5
# if it goes beyond 5, stop sending requests, call this every 3 seconds

# bus scheduled and expected randomly get pushed back!!!


# 2021-10-23T18:05:36-05:00 -> 18:05:36 for 'soft', datetime for 'hard'
def time_converter(time, soft_hard):
    split_1 = time.split("T")
    split_2 = split_1[1].split("-")
    if soft_hard == "hard":
        split_3 = split_2[0].split(':')
        return datetime.datetime.now().replace(hour = int(split_3[0]), minute = int(split_3[1]), second = int(split_3[2]), microsecond = 0)
    elif soft_hard == "soft":
        return str(split_2[0])

# Returns a list of all buses, with different vehicle_id's, currently heading to this stop_id.
# When a bus leaves, they stop appearing in requests, and therefore the list everytime it's called.
def arrival_queue(stop_id):
    list = []
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=" + stop_id)
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in list: # Eliminating any dupes, leaving them for next time they're called
            list.append(str(response.json()["departures"][i]["headsign"]))
    print(list) # Eliminate this print later
    return list

#  key:'headsign', 0:'route_id', 1:'origin', 2:'destination', 3:'scheduled', 4:'expected', 5:'expected_mins', dictionary with info about current buses with same stop_id
def stop_id_info(stop_id):
    bus_dict = {}
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=" + stop_id)
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in bus_dict.keys(): # Eliminating any dupes, leaving them for next time they're called
            bus_dict[str(response.json()["departures"][i]["headsign"])] = [str(response.json()["departures"][i]["route"]["route_id"]), str(response.json()["departures"][i]["origin"]["stop_id"]), str(response.json()["departures"][i]["destination"]["stop_id"]), str(response.json()["departures"][i]["scheduled"]), str(response.json()["departures"][i]["expected"]), str(response.json()["departures"][i]["expected_mins"])]
    return bus_dict
    print(bus_dict) # Eliminate this print later

# Returns time difference when given stop_id, the bus_headsign, and the index that determines whether it is scheduled or expected
def time_difference(stop_id, bus_headsign, index):
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=" + stop_id)
    current_time = time_converter(response.json()["time"], 'hard')
    current_sc_ex = time_converter(stop_id_info(stop_id)[bus_headsign][index], 'hard')
    return (current_time - current_sc_ex).total_seconds()
                
# Outcome of whether current several buses came before or after the scheduled or expected time -> 3 for scheduled, 4 or expected
def bulk_difference(stop_id, sc_ex):
    results = []
    scheduled_times = {}
    expected_times = {}
    bus_headsign = arrival_queue(stop_id)
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=" + stop_id)
    for i in range(0, 1):
        # grab first element from the bus_headsign, gather info on scheduled and expected
        # send requests constantly when time difference in expected is -120 to 420 -> -2 to 7 minutes
        # while sending the requests, if bus disappears from list, then stop sending requests and keep the time difference value
        # pop the first bus element from the bus_headsign and go on to the next one until done
        # current_expected = time_converter(stop_id_info(stop_id)[bus_headsign[i]][4], 'hard')
        
        while (True):
            if -120 <= int(time_difference(stop_id, bus_headsign[i], sc_ex)) and int(time_difference(stop_id, bus_headsign[i], sc_ex)) <= 420:
                print(int(time_difference(stop_id, bus_headsign[i], sc_ex)))
                if bus_headsign[i] not in arrival_queue(stop_id):
                    results.append(int(time_difference(stop_id, bus_headsign[i], sc_ex)))
                    break
                else:
                    time.sleep(3)
            else:
                time.sleep(30)
            
    print(results)
        
        
# Outcome of whether the bus came before or after the scheduled or expected time
def indiv_difference(headsign, stop_id, sc_or_ex):
    print('')

print(time_difference('IU:1', '220N Illini', 4))
bulk_difference('IU:1', 3)
stop_id_info('IU:1')

-693.0
['130N Silver', '50E Green Lsq', '220N Illini', '120E Teal Orchard Downs']


ConnectionError: HTTPSConnectionPool(host='developer.cumtd.com', port=443): Max retries exceeded with url: /api/v2.2/json/getdeparturesbystop?key=abcda32ab81b438fb628a908c94746bb&stop_id=IU:1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8a306fd070>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [324]:
response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=" + 'IU:1')
response.json()

{'time': '2021-10-24T19:23:49-05:00',
 'new_changeset': True,
 'status': {'code': 200, 'msg': 'ok'},
 'rqst': {'method': 'GetDeparturesByStop', 'params': {'stop_id': 'IU:1'}},
 'departures': [{'stop_id': 'IU:1',
   'headsign': '220N Illini',
   'route': {'route_color': '5a1d5a',
    'route_id': 'ILLINI EVENING SUNDAY',
    'route_long_name': 'Illini Evening Sunday',
    'route_short_name': '220',
    'route_text_color': 'ffffff'},
   'trip': {'trip_id': '[@124.0.92263401@][1484328944432]/83__I6_UISU',
    'trip_headsign': 'Lincoln & Killarney',
    'route_id': 'ILLINI EVENING SUNDAY',
    'block_id': 'I6 UISU',
    'direction': 'North',
    'service_id': 'I6 UISU',
    'shape_id': '[@124.0.92263401@]220N ILLINI 10'},
   'vehicle_id': '1608',
   'origin': {'stop_id': 'PAR:2'},
   'destination': {'stop_id': 'LNCLNKLRNY:1'},
   'is_monitored': True,
   'is_scheduled': True,
   'is_istop': True,
   'scheduled': '2021-10-24T19:26:00-05:00',
   'expected': '2021-10-24T19:25:52-05:00',
   'ex

In [ ]:
    for i in range(0, len(bus_headsign)): # this may be a mistake since scheduled and expected times randomly get pushed back
        scheduled_times[bus_headsign[i]] = time_converter(str(response.json()["departures"][i]["scheduled"]), 'hard')
        expected_times[bus_headsign[i]] = time_converter(str(response.json()["departures"][i]["expected"]), 'hard') 
        
    amongus = time_converter(response.json()["time"], 'hard') - scheduled_times['50E Green']
    print(time_converter(response.json()["time"], 'hard')) # there are several bus stops with the same headsigns, it is grabbing the last one and not several, maybe consider using vehicle_id
    print(scheduled_times['50E Green'])
    print(amongus.total_seconds()) # We have to take note of this value until it response indicates that the bus is no longer waiting, but, does it disappear when expected is reached or until it arrives?